In [2]:
# ---------------------------------------------
# 1️⃣ Install Supabase client (run once)
# ---------------------------------------------
!pip install supabase python-dotenv pandas

In [3]:
# ---------------------------------------------
# 2️⃣ Import libraries
# ---------------------------------------------
import pandas as pd
from supabase import create_client, Client
from dotenv import load_dotenv
import os

In [4]:
# ---------------------------------------------
# 3️⃣ Load environment variables from .env
# ---------------------------------------------
load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

print("Loaded .env successfully!")

Loaded .env successfully!


In [5]:
# ---------------------------------------------
# 4️⃣ Initialize Supabase client
# ---------------------------------------------
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print("Connected to Supabase!")

Connected to Supabase!


In [7]:
# ---------------------------------------------
# 5️⃣ Load processed dataset
# ---------------------------------------------
import pandas as pd

df = pd.read_csv("data/processed/churn_processed.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group,monthly_charge_segment,has_internet_service,is_multi_line_user,contract_type_code
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,Yes,Electronic check,29.85,29.85,No,0-12,Low,1,0,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,Mailed check,56.95,1889.50,No,25-36,Medium,1,0,1
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,Yes,Mailed check,53.85,108.15,Yes,0-12,Medium,1,0,0
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,Bank transfer (automatic),42.30,1840.75,No,37-48,Medium,1,0,1
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Yes,Electronic check,70.70,151.65,Yes,0-12,High,1,0,0


In [ ]:
# =========================================================
# ✅ Upload processed customer churn dataset to Supabase
# Table: customer_churn_v2 (all lowercase columns)
# =========================================================

import os
import pandas as pd
from supabase import create_client
from dotenv import load_dotenv

# -----------------------------------------
# 1️⃣ Load environment variables and init client
# -----------------------------------------
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

# -----------------------------------------
# 2️⃣ Load processed CSV
# -----------------------------------------
processed_csv_path = "data/processed/churn_processed.csv"

if not os.path.exists(processed_csv_path):
    raise FileNotFoundError(f"❌ Processed CSV not found at {processed_csv_path}")

df = pd.read_csv(processed_csv_path)
print(f"📊 Loaded {len(df)} rows from {processed_csv_path}")

# -----------------------------------------
# 3️⃣ Rename columns to lowercase to match table
# -----------------------------------------
df_upload = df.rename(columns=str.lower)

# Define the exact columns of the Supabase table
columns_to_upload = [
    "customerid", "gender", "seniorcitizen", "partner", "dependents",
    "tenure", "phoneservice", "internetservice", "contract",
    "monthlycharges", "totalcharges", "churn"
]

# Filter dataframe to match table columns
df_upload = df_upload[columns_to_upload]

# -----------------------------------------
# 4️⃣ Convert NaN to None for proper NULL handling
# -----------------------------------------
df_upload = df_upload.where(pd.notnull(df_upload), None)

# -----------------------------------------
# 5️⃣ Convert to list of dictionaries for Supabase insert
# -----------------------------------------
records = df_upload.to_dict(orient="records")

# -----------------------------------------
# 6️⃣ Optional: refresh schema cache
# -----------------------------------------
supabase.table("customer_churn_v2").select("*").execute()

# -----------------------------------------
# 7️⃣ Upload to Supabase
# -----------------------------------------
response = supabase.table("customer_churn_v2").insert(records).execute()

print("✅ Upload completed!")
print(f"Rows uploaded: {len(records)}")

NameError: name 'churn_processed' is not defined